In [1]:
!pip install ..

Processing /home/jupyter/code/szaboildi/uk-pol-speech-classifier
  Preparing metadata (setup.py) ... done
  Created wheel for polclassifier: filename=polclassifier-0.0.1-py3-none-any.whl size=13693 sha256=49b0722363c95ef45ce510d3392d4100fbfb96f9b9ed2cda8eb8803497e9fb4f
  Stored in directory: /home/jupyter/.cache/pip/wheels/62/8c/bc/f3bb23466cf275ad76727c3996b011e2321ab07a4eee1b8593
Successfully built polclassifier
  Attempting uninstall: polclassifier
    Found existing installation: polclassifier 0.0.1
    Uninstalling polclassifier-0.0.1:
      Successfully uninstalled polclassifier-0.0.1


In [2]:
import os
from colorama import Fore, Style
import pandas as pd

from sklearn.model_selection import train_test_split
from polclassifier.ml_logic.preprocessing import *
from polclassifier.ml_logic.models import *
from polclassifier.ml_logic.registry import *
from polclassifier.params import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2024-05-15 07:57:05.768847: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 07:57:06.469087: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environ

In [3]:
def load_speeches(min_word_count=400, sample_size=1000, parties_to_exclude=[], speeches_per_party = 20):
    print("function")
    # Load data from feather file
    raw_data_path = os.path.join(
            LOCAL_PATH, "raw_data", "Corp_HouseOfCommons_V2.feather")
    data = pd.read_feather(raw_data_path)

    # Filter and clean data
    data = clean_data(df=data, min_word_count=min_word_count, sample_size=sample_size, parties_to_exclude=parties_to_exclude)

    # Split the data into training and testing sets
    data_train, data_test = train_test_split(data, test_size=0.2, random_state=42, stratify=data["party"])

    # Undersample data_test
    grouped_data = data_test.groupby('party')
    smaller_data_test = []
    for party, group in grouped_data:
        # Select randomly 20 speeches per party
        sampled_group = group.sample(n=speeches_per_party, random_state=42)
        # Add selected speeches to list
        smaller_data_test.append(sampled_group)

    df = pd.concat(smaller_data_test)
    path = os.path.join(LOCAL_PATH, "processed_data", "smaller_data_test.csv")
    df.to_csv(path, index=False)

    return

In [5]:
load_speeches()

function


/opt/conda/lib/python3.10/site-packages/polclassifier/ml_logic/preprocessing.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["word_n_full"] = df.apply(lambda row: len(row["text"].strip().split()), axis=1)


In [7]:
df = pd.read_csv(os.path.join(LOCAL_PATH, "processed_data", "smaller_data_test.csv"))

In [17]:
df.party.value_counts()

party
Con           20
DUP           20
Lab           20
LibDem        20
PlaidCymru    20
SNP           20
UUP           20
Name: count, dtype: int64

In [28]:
party_data = df[df['party'] == "Lab"].reset_index(drop=Tru)
party_data

,index,speaker,party,text,word_n_full
0,40,Phil Hope,Lab,The regulations governing the leakage of oil f...,870
1,41,Norman Godman,Lab,"I promise that I shall be brief. Last week, I ...",473
2,42,Mike Gapes,Lab,I shall not recite a litany of individual case...,1187
3,43,Joan Walley,Lab,"I will be brief, because I know that many hon....",926
4,44,Mo Mowlam,Lab,"No, I shall not give way, because I have only ...",1309
5,45,Dawn Primarolo,Lab,I am not ruling out further alignment. It is s...,503
6,46,Ian Lucas,Lab,We consider and assess individual applications...,1085
7,47,Alan Whitehead,Lab,I have said in the House and on previous occas...,710
8,48,Ian Gibson,Lab,I thank the hon. Gentleman for that interventi...,495
9,49,Alan Whitehead,Lab,Neither have I. As I granted the hon. Member f...,428


In [26]:
import random

def get_speech(data, party: str):

    # Filter data based on party
    party_data = data[data['party'] == party].reset_index(drop_)

    # Check if there are speeches available for the selected party
    if party_data.empty:
        return {"error": "No speeches found for the selected party."}

    # Select a random speech from the filtered data
    selected_speech = random.choice(party_data['text'])
    return dict(speech = selected_speech)

In [27]:
get_speech(data=df, party="Lab")

{'speech': "No, I shall not give way, because I have only 10 minutes. On the social chapter, it is no use the hon. and learned Gentleman saying that the Government can introduce the changes. We have waited 12 years. To take one simple example, working women in this country do not have the maternity benefits that women working in other European countries consider a basic right. We have waited 12 years - You don't get pregnant, sweetheart. We have waited 12 years for that change, and it has not come. What the Prime Minister said in the Chamber yesterday was inaccurate-we are arguing not about the length of maternity leave, but about maternity benefits. As female Conservative hon. Members know, about 50 per cent. of women who apply for maternity benefit do not get it, because of the stringent qualifications. The social chapter could achieve such basic rights for women. That is why we are pushing for it, and that is why the other 11 countries are in a different position from this country. 